In [ ]:
import json
import numpy as np
import pandas as pd
import requests

from langchain.chains import StuffDocumentsChain, RetrievalQA, LLMChain, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.llms import Ollama, BaseLLM
from langchain.schema import Document, Generation, LLMResult
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_community.llms import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from pathlib import Path
from tqdm import tqdm
from glob import glob

class LocalOllamaLLM(BaseLLM):
    api_url : str
    def _generate(self, prompt, stop):
        response = requests.post(f"{self.api_url}/api/generate", json={"model": "llama3.1", "prompt": str(prompt) })
        response.raise_for_status()
        response_text=''.join([json.loads(line)['response'] for line in response.text.splitlines()])
        generations=[]
        generations.append([Generation(text=response_text)])
        return LLMResult(generations=generations)


    def _llm_type(self):
        return "local"  # Or whatever type is appropriate for your local setup

llm = LocalOllamaLLM(api_url="http://127.0.0.1:11434")

embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vector_store = Chroma(embedding_function=embedder, persist_directory="./chroma_db")


system_prompt = (
    " Répondez à la question posée "
    " Utilisez le contexte (sélection des meilleurs paragraphes liés à la question) donné pour répondre à la question "
    " Si la réponse ne se trouve pas dans le contexte, répondez par 'Je ne sais pas'"
    " Contexte : {context}  "
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)


def search_and_invoke_llm(vector_store,index,query,k=5):
    if k==0:
        print(f"bug with {index}")
        return None
    else:
        pass
    try:
        retriever=vector_store.as_retriever(
        search_kwargs={
                "k": k, 
                "filter": {'index': index}
            }
        )
        chain = create_retrieval_chain(retriever, question_answer_chain)
        result=chain.invoke({"input": query})
        return result
    except:
        search_and_invoke_llm(vector_store,index,query,k=k-1)
    return None

In [ ]:
QUESTIONS=["Quelles sont les règles de majoration des heures supplémentaires ?",
          "De combien est le contingent annuel d'heures supplémentaires ?",
          "Quelle est la durée maximale hebdomadaire de travail ?",
          "Quelles les contreparties en repos ?",
          "De combien est le repos compensateur obligatoire?"]

In [ ]:
file="./10p_accords_publics_et_thematiques_240815_sub_heures_supp.parquet"
df=pd.read_parquet(file)

In [ ]:
list_of_df=[]
Path("results").mkdir(parents=True, exist_ok=True)
for index, row in df.iterrows():
    dict_answer=dict()
    answer=""
    for Q0 in QUESTIONS:
        if ans:=search_and_invoke_llm(vector_store,index,Q0,k=2):
            answer_txt=ans['answer']
            answer+=f"{ans['input']} {answer_txt} \n"
    with open(f"results/{index}.answer","w") as file:
        file.write(answer)
    
    with open(f"results/{index}.context","w") as file:
        try:
            file.write("\n-----\n".join(list(map(lambda x : x.page_content,ans['context']))))
        except:
            file.write("")